# Fine-tuning with QLoRA (Colab Version)

This notebook demonstrates how to fine-tune a language model using QLoRA (Quantized Low-Rank Adaptation) for text summarization tasks in Google Colab.

## 1. Setup Environment

First, we'll mount Google Drive and install required packages.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set up project directory
import os

# Change this to your Google Drive project path
PROJECT_DIR = "/content/drive/MyDrive/text_summarization_project"
DATA_DIR = os.path.join(PROJECT_DIR, "data")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed")
MODEL_DIR = os.path.join(PROJECT_DIR, "models")
QLORA_MODEL_DIR = os.path.join(MODEL_DIR, "finetuned_qlora")

# Create necessary directories
os.makedirs(QLORA_MODEL_DIR, exist_ok=True)

# Install required packages
!pip install -q transformers datasets accelerate bitsandbytes peft trl
!pip install -q torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2
!pip install -q evaluate rouge-score numpy pandas matplotlib seaborn

## 2. Load and Prepare Data

Load the preprocessed data from the previous notebook.

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load the preprocessed data
train_df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "train.csv"))
val_df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "validation.csv"))
test_df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "test.csv"))

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Create dataset dictionary
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

## 3. Initialize Model and Tokenizer

Load the base model and configure it for QLoRA fine-tuning.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
import torch

# Model configuration
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

## 4. Configure QLoRA

Set up the QLoRA configuration for fine-tuning.

In [ ]:
from peft import LoraConfig

# LoRA configuration
lora_config = LoraConfig(
    r=8,                     # Rank
    lora_alpha=32,          # Alpha scaling
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA config to the model
model = get_peft_model(model, lora_config)

## 5. Training Configuration

Set up the training arguments and prepare for training.

In [ ]:
from transformers import TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir=QLORA_MODEL_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    logging_steps=10,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none"
)

## 6. Start Training

Initialize the trainer and start the fine-tuning process.

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Start training
trainer.train()

## 7. Save and Upload Model

Save the trained model and upload it back to Google Drive.

In [ ]:
# Save the model
trainer.save_model(QLORA_MODEL_DIR)

# Upload the model back to Drive
!zip -r /content/qlora_model.zip $QLORA_MODEL_DIR
from google.colab import files
files.download('/content/qlora_model.zip')